# March Madness Monte Carlo Simulation

Use information to try to predict the 2023 March Madness Men's and Women's Basketball Championships.

## Configure Notebook and Download Competition Items

In [2]:
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [3]:
from pathlib import Path
import os

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/march-machine-learning-mania-2023')
    !pip install -Uqq fastai
else:
    import zipfile,kaggle
    path = Path('march-machine-learning-mania-2023')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

## Load and Preview Provided Data

In [4]:
import pandas as pd

cities = pd.read_csv(path/'Cities.csv')
conferences = pd.read_csv(path/'Conferences.csv')
m_conf_tourney_games = pd.read_csv(path/'MConferenceTourneyGames.csv')
m_game_cities = pd.read_csv(path/'MGameCities.csv')
m_massey_ordinals = pd.read_csv(path/'MMasseyOrdinals_thru_Season2023_Day128.csv')
m_tourney_compact_results = pd.read_csv(path/'MNCAATourneyCompactResults.csv')
m_tourney_detailed_results = pd.read_csv(path/'MNCAATourneyDetailedResults.csv')
m_tourney_seed_round_slots = pd.read_csv(path/'MNCAATourneySeedRoundSlots.csv')
m_tourney_seeds = pd.read_csv(path/'MNCAATourneySeeds.csv')
m_tourney_slots = pd.read_csv(path/'MNCAATourneySlots.csv')
m_reg_season_compact_results = pd.read_csv(path/'MRegularSeasonCompactResults.csv')
m_reg_season_detailed_results = pd.read_csv(path/'MSeasons.csv')
m_secondary_tourney_compact_results = pd.read_csv(path/'MSecondaryTourneyCompactResults.csv')
m_secondary_tourney_teams = pd.read_csv(path/'MSecondaryTourneyTeams.csv')
m_team_coaches = pd.read_csv(path/'MTeamCoaches.csv')
m_team_conferences = pd.read_csv(path/'MTeamConferences.csv')
m_teams = pd.read_csv(path/'MTeams.csv')
m_team_spellings = pd.read_csv(path/'MTeamSpellings.csv', encoding='latin-1')
samp_submission = pd.read_csv(path/'SampleSubmission2023.csv')
w_game_cities = pd.read_csv(path/'WGameCities.csv')
w_tourney_compact_results = pd.read_csv(path/'WNCAATourneyCompactResults.csv')
w_tourney_detailed_results = pd.read_csv(path/'WNCAATourneyDetailedResults.csv')
w_tourney_seeds = pd.read_csv(path/'WNCAATourneySeeds.csv')
w_tourney_slots = pd.read_csv(path/'WNCAATourneySlots.csv')
w_reg_season_compact_results = pd.read_csv(path/'WRegularSeasonCompactResults.csv')
w_reg_season_detailed_results = pd.read_csv(path/'WRegularSeasonDetailedResults.csv')
w_seasons = pd.read_csv(path/'WSeasons.csv')
w_team_conferences = pd.read_csv(path/'WTeamConferences.csv')
w_teams = pd.read_csv(path/'WTeams.csv')
w_team_spellings = pd.read_csv(path/'WTeamSpellings.csv', encoding='latin-1')

In [7]:
m_teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2023
1,1102,Air Force,1985,2023
2,1103,Akron,1985,2023
3,1104,Alabama,1985,2023
4,1105,Alabama A&M,2000,2023


# Process Overview

Use ML to submit predictions related to the [Kaggle 2023 March Madness Competition](https://www.kaggle.com/competitions/march-machine-learning-mania-2023).

Process steps:
1. Evaluate data and build a classifier to predict probability a team wins in a head-to-head matchup
2. Use model within a Monte Carlo simulation to simulate the tournament


In [8]:
m_teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2023
1,1102,Air Force,1985,2023
2,1103,Akron,1985,2023
3,1104,Alabama,1985,2023
4,1105,Alabama A&M,2000,2023


In [9]:
m_team_spellings.head()

,TeamNameSpelling,TeamID
0,a&m-corpus chris,1394
1,a&m-corpus christi,1394
2,abilene chr,1101
3,abilene christian,1101
4,abilene-christian,1101
